## Import packages and data

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

#import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

2022-12-20 18:09:22.889669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df_item=pd.read_csv('/Users/philippebeliveau/Desktop/Bureau/Notebook_Jupyter_&_R/I2A2 Notebook copy/Deep AR M1 CHALLENGE /datah-m1-challange/meta-item.csv')
df_store=pd.read_csv('/Users/philippebeliveau/Desktop/Bureau/Notebook_Jupyter_&_R/I2A2 Notebook copy/Deep AR M1 CHALLENGE /datah-m1-challange/meta-store.csv')
df_sales=pd.read_csv('/Users/philippebeliveau/Desktop/Bureau/Notebook_Jupyter_&_R/I2A2 Notebook copy/Deep AR M1 CHALLENGE /datah-m1-challange/sales.csv')

df_submission=pd.read_csv('submission_sample.csv')
df_submission[['DATE', 'S100', 'I100', 'C100', 'C101']] = df_submission['ID'].str.split('_',expand=True)
df_submission['S100']=df_submission['S100'].astype(int)
df_submission['I100']=df_submission['I100'].astype(int)

In [4]:
def preprocess(df):
    
    df[['year', 'month', 'day']] = df['DATE'].str.split('-',expand=True)
    
    for value in [ 'S100', 'I100', 'C100', 'C101','month', 'year', 'day']:        
        df[value] = df[value].astype(str).astype(int)
    
    df = df.merge(df_item,on=['I100'])
    df = df.merge(df_store,on=['S100'])
    

    df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime('%Y-%m-%d')    
    return df

In [5]:
df_sales= preprocess(df_sales)
df_submission =preprocess(df_submission)
df_submission.head(5)

,ID,QTT,DATE,S100,I100,C100,C101,year,month,day,I101,I102,I103,S101,S102,S103
0,2021-10-03_0_1_12_140,-1,2021-10-03,0,1,12,140,2021,10,3,2,1,1,1,17,10
1,2021-10-03_0_1_12_164,-1,2021-10-03,0,1,12,164,2021,10,3,2,1,1,1,17,10
2,2021-10-03_0_1_12_339,-1,2021-10-03,0,1,12,339,2021,10,3,2,1,1,1,17,10
3,2021-10-03_0_1_13_128,-1,2021-10-03,0,1,13,128,2021,10,3,2,1,1,1,17,10
4,2021-10-03_0_1_14_164,-1,2021-10-03,0,1,14,164,2021,10,3,2,1,1,1,17,10


## Divide dataset

### Divide X and Y

In [6]:
features_names =['S100', 'I100', 'C100', 'C101', 'month', 'year', 'day', 'I101', 'I102', 'I103', 'S101', 'S102', 'S103']
features_names_qtt =['QTT', 'S100', 'I100', 'C100', 'C101', 'month', 'year', 'day', 'I101', 'I102', 'I103', 'S101', 'S102', 'S103']
X_lstm=df_sales[features_names_qtt]
#Training
X = df_sales[features_names]
Y = df_sales['QTT']

#Submission
x_sub= df_submission[features_names]
y_sub=df_submission['QTT']

In [7]:
#Scale train data 
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
data_scaled= scaler.fit_transform(X_lstm)
features = data_scaled
Y_scaled = data_scaled[:,0]

In [8]:
Y_scaled.shape

(2749369,)

In [9]:
#Scale submission Data
submission_scaled= scaler.fit_transform(x_sub)
sub_features = submission_scaled
y_sub = submission_scaled[:,0]

### For LGBM

In [10]:
from sklearn.model_selection import TimeSeriesSplit

x_train, x_test, y_train,y_test = train_test_split(X,Y, test_size=0.2, random_state=42)  
X_validation, X_test, y_validation, y_test=train_test_split(x_test, y_test, test_size=0.5, random_state=42)

### For LSTM

In [11]:
sequence_len = 32
batch_size= 64
features_len = 14

X_train, X_test, y_train, y_test = train_test_split(features, Y_scaled, test_size=0.2, random_state=42)  
X_valid= sub_features
y_valid=y_sub

train_generator = TimeseriesGenerator(X_train, y_train, length=sequence_len, batch_size=batch_size)
test_generator= TimeseriesGenerator(X_test, y_test, length=sequence_len, batch_size=batch_size)
valid_generator= TimeseriesGenerator(X_valid, y_valid, length=sequence_len, batch_size=batch_size)

# LGBM

In [ ]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['rmse'],
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    "max_depth": 8,
    "num_leaves": 64,  
    "max_bin": 512,
    "num_iterations": 100
}

In [ ]:
gbm = lgb.LGBMRegressor(**hyper_params)

In [ ]:
gbm.fit(x_train, y_train, eval_set=[(X_validation, y_validation)], setval_metric='rmse', early_stopping_rounds=30, verbose=50)

# LSTM

In [12]:
features, Y_scaled =train_generator[0]

print(len(train_generator))
print("batch x shape:", X_lstm.shape)
print('batch y shape:', Y.shape)
print(train_generator[0])

34367
batch x shape: (2749369, 14)
batch y shape: (2749369,)
(array([[[0.        , 0.2       , 0.21052632, ..., 1.        ,
         0.47368421, 0.7       ],
        [0.        , 1.        , 0.86315789, ..., 1.        ,
         0.        , 1.        ],
        [0.00144928, 0.7       , 0.52631579, ..., 1.        ,
         0.57894737, 0.8       ],
        ...,
        [0.        , 0.8       , 0.2       , ..., 1.        ,
         0.68421053, 0.6       ],
        [0.00144928, 0.46666667, 0.49473684, ..., 1.        ,
         0.15789474, 0.2       ],
        [0.        , 0.66666667, 0.11578947, ..., 1.        ,
         0.78947368, 1.        ]],

       [[0.        , 1.        , 0.86315789, ..., 1.        ,
         0.        , 1.        ],
        [0.00144928, 0.7       , 0.52631579, ..., 1.        ,
         0.57894737, 0.8       ],
        [0.        , 0.66666667, 0.18947368, ..., 1.        ,
         0.78947368, 1.        ],
        ...,
        [0.00144928, 0.46666667, 0.49473684, .

In [13]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(32, input_shape=(sequence_len, features_len), return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(keras.layers.LSTM(32, return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.Dropout(0.5))
model.add(keras.layers.LSTM(32, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1))

model.summary()

2022-12-20 18:09:39.890282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32, 32)            6016      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32, 32)            8320      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32, 32)            0         
                                                                 
 dropout (Dropout)           (None, 32, 32)            0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0

In [14]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=2,
                                                    mode='min')

model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(),
              metrics=[tf.metrics.MeanAbsoluteError()])


Simply to long to train - that's why the error

In [15]:
history = model.fit_generator(train_generator, epochs=3,
                    validation_data=test_generator,
                    shuffle=False,
                    callbacks=[early_stopping])

Epoch 1/3


/var/folders/zx/_8wgnq2s1vg17f0_9lwx4swh0000gn/T/ipykernel_1427/3061854500.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=3,


34367/34367 [==============================] - 3047s 89ms/step - loss: 2.5498e-05 - mean_absolute_error: 0.0026 - val_loss: 2.2149e-05 - val_mean_absolute_error: 0.0026
Epoch 2/3
34367/34367 [==============================] - 1071s 31ms/step - loss: 2.3149e-05 - mean_absolute_error: 0.0024 - val_loss: 2.2154e-05 - val_mean_absolute_error: 0.0026
Epoch 3/3
34367/34367 [==============================] - 949s 28ms/step - loss: 2.3145e-05 - mean_absolute_error: 0.0024 - val_loss: 2.2146e-05 - val_mean_absolute_error: 0.0026


In [17]:
model.evaluate_generator(test_generator, verbose=0)  

/var/folders/zx/_8wgnq2s1vg17f0_9lwx4swh0000gn/T/ipykernel_1427/748299794.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(test_generator, verbose=0)


[2.2145539332996123e-05, 0.002582989865913987]

In [18]:
#Predict on test set
predictions_test=model.predict_generator(test_generator)

/var/folders/zx/_8wgnq2s1vg17f0_9lwx4swh0000gn/T/ipykernel_1427/839825740.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_test=model.predict_generator(test_generator)


In [22]:
# Concat my pred with the actual 
df_pred = pd.concat([pd.DataFrame(predictions_test), pd.DataFrame(X_test[:,1:][sequence_len:])], axis=1)

In [23]:
df_pred

,0,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.002434,0.000000,0.494737,0.75,0.217514,0.636364,0.25,0.366667,0.0,0.5,1.0,1.0,0.894737,1.0
1,0.002434,0.600000,0.705263,0.75,0.830508,0.636364,0.25,0.366667,1.0,1.0,1.0,1.0,0.368421,0.1
2,0.002433,0.966667,0.021053,0.65,0.042373,0.272727,0.00,0.966667,0.4,0.5,0.5,1.0,0.684211,0.6
3,0.002433,0.233333,0.368421,0.25,1.000000,0.909091,0.50,0.533333,0.0,0.5,0.5,1.0,0.210526,1.0
4,0.002433,0.366667,0.873684,0.65,0.810734,0.545455,0.50,0.666667,0.6,1.0,1.0,0.0,0.631579,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549837,0.002433,0.933333,0.231579,0.65,1.000000,0.272727,1.00,0.333333,0.4,0.5,1.0,1.0,1.000000,0.4
549838,0.002433,0.433333,0.452632,0.20,0.214689,0.181818,0.25,0.333333,0.0,0.0,0.5,1.0,0.157895,0.2
549839,0.002433,0.600000,0.568421,0.70,1.000000,0.727273,0.00,0.300000,0.0,0.0,1.0,1.0,0.368421,0.1
549840,0.002433,0.566667,0.536842,0.70,0.217514,0.454545,0.25,0.300000,0.0,0.5,1.0,1.0,0.736842,1.0
